In [1]:
from nltk.lm import MLE
from nltk import FreqDist
from nltk.util import ngrams
from nltk.lm import WittenBellInterpolated as WB
from nltk.lm import KneserNeyInterpolated as kn
from nltk.lm import Laplace
from nltk.lm import Lidstone
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
data=open(r"C:\Users\Dell\Videos\NLP\speeches.txt",'r',encoding='utf-8')

In [3]:
text=data.read()

In [4]:
sent=sent_tokenize(text)

In [5]:
#A list of sentences in the text
sent

['\ufeffSPEECH 1\n\n\n...Thank you so much.',
 "That's so nice.",
 "Isn't he a great guy.",
 "He doesn't get a fair press; he doesn't get it.",
 "It's just not fair.",
 "And I have to tell you I'm here, and very strongly here, because I have great respect for Steve King and have great respect likewise for Citizens United, David and everybody, and tremendous resect for the Tea Party.",
 'Also, also the people of Iowa.',
 'They have something in common.',
 'Hard-working people.',
 'They want to work, they want to make the country great.',
 'I love the people of Iowa.',
 "So that's the way it is.",
 'Very simple.',
 'With that said, our country is really headed in the wrong direction with a president who is doing an absolutely terrible job.',
 "The world is collapsing around us, and many of the problems we've caused.",
 'Our president is either grossly incompetent, a word that more and more people are using, and I think I was the first to use it, or he has a completely different agenda th

In [55]:
data.close()

In [7]:
#cleaning text - lowering cases, removing punctuations, padding start and end tokens
import re
for i in range(len(sent)):
    sentence=sent[i]
    sentence=sentence.lower()
    sentence=re.sub(r'[^a-zA-Z\'\s]', "",sentence)
    sentence="<s> "+sentence+" </s>"
    sent[i]=sentence

In [8]:
sent

['<s> speech \n\n\nthank you so much </s>',
 "<s> that's so nice </s>",
 "<s> isn't he a great guy </s>",
 "<s> he doesn't get a fair press he doesn't get it </s>",
 "<s> it's just not fair </s>",
 "<s> and i have to tell you i'm here and very strongly here because i have great respect for steve king and have great respect likewise for citizens united david and everybody and tremendous resect for the tea party </s>",
 '<s> also also the people of iowa </s>',
 '<s> they have something in common </s>',
 '<s> hardworking people </s>',
 '<s> they want to work they want to make the country great </s>',
 '<s> i love the people of iowa </s>',
 "<s> so that's the way it is </s>",
 '<s> very simple </s>',
 '<s> with that said our country is really headed in the wrong direction with a president who is doing an absolutely terrible job </s>',
 "<s> the world is collapsing around us and many of the problems we've caused </s>",
 '<s> our president is either grossly incompetent a word that more and m

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
#splitting dataset
train,test=train_test_split(sent,shuffle=False,test_size=0.2)

In [161]:
train

['<s> speech \n\n\nthank you so much </s>',
 "<s> that's so nice </s>",
 "<s> isn't he a great guy </s>",
 "<s> he doesn't get a fair press he doesn't get it </s>",
 "<s> it's just not fair </s>",
 "<s> and i have to tell you i'm here and very strongly here because i have great respect for steve king and have great respect likewise for citizens united david and everybody and tremendous resect for the tea party </s>",
 '<s> also also the people of iowa </s>',
 '<s> they have something in common </s>',
 '<s> hardworking people </s>',
 '<s> they want to work they want to make the country great </s>',
 '<s> i love the people of iowa </s>',
 "<s> so that's the way it is </s>",
 '<s> very simple </s>',
 '<s> with that said our country is really headed in the wrong direction with a president who is doing an absolutely terrible job </s>',
 "<s> the world is collapsing around us and many of the problems we've caused </s>",
 '<s> our president is either grossly incompetent a word that more and m

In [162]:
#pickle
import pickle 
train_dump=open(r"C:\Users\Dell\Videos\NLP\traindump.txt",'wb')
test_dump=open(r"C:\Users\Dell\Videos\NLP\testdump.txt",'wb')
pickle.dump(train,train_dump)
pickle.dump(test,test_dump)
train_dump.close()
test_dump.close()

In [12]:
vocab=list(set((' '.join(sent)).split()))

In [13]:
# vocab_dump
vocab_dump=open(r"C:\Users\Dell\Videos\NLP\traindump.txt",'wb')
pickle.dump(vocab,vocab_dump)
vocab_dump.close()
    
    

In [137]:
len(vocab)

6071

In [12]:
import nltk

In [13]:
from nltk import FreqDist

In [271]:
#ngram function to find MLE distributions and perplexity. freq_n1 is to be ignored, it is for further functions.
def ngram(n,train,test,vocab):
    import math
    if n>1:
        train_n=[]
        train_n1=[]
        test_n=[]
        test_n1=[]
        for j in train:
            train_n.extend(list(ngrams(j.split(),n)))
            train_n1.extend(list(ngrams(j.split(),n-1)))
        for j in test:
            test_n.extend(list(ngrams(j.split(),n)))
            test_n1.extend(list(ngrams(j.split(),n-1)))
        freq_n=dict(FreqDist(train_n))
        freq_n1=dict(FreqDist(train_n1))
        #test_n1 = list(ngrams(test_tokens,n-1))
        for i in test_n:
            if i not in freq_n.keys():
                freq_n[i]=0
        V=len((vocab))
        for i in test_n1:
            #print(i)
            if i not in freq_n1.keys():
                freq_n1[i]=0
#             else:
#                 freq_n1[i]=V
            #print(i,freq_n1[i])
        for i in freq_n.keys():
            freq_n[i]+=1
        for i in freq_n1.keys():
            freq_n1[i]+=V
            #print(i,freq_n1[i])
        prob_n={}
        #print(freq_n.keys())
        #time.sleep(3)
        #print(freq_n1)
        for i in freq_n.keys():
            #print(train_n1)
            #time.sleep(2)
            #print()
            #print(i,freq_n[i],i[0:n-1],freq_n1[i[0:n-1]],train_n1.index(i[:n-1]))
            prob_n[i]=(freq_n[i])/(freq_n1[i[0:n-1]])
        perplexityn=0 
        #print(prob_n)
        for i in test_n:
            #if i in prob_n.keys():
            perplexityn+=math.log(prob_n[i],2)
            #else:
                #perplexityn+=math.log(1/(freq_n1[i[0:n-1]]),2) 
        perplexityn/=len(test_n)
        perplexityn=(-perplexityn)
        perplexityn=2**(perplexityn)
        #print(perplexityn)
        
        #test_n = list(ngrams(test_tokens,n))
    elif n==1:
        freq_n1={}
        train_n=[]
        for j in train:
            train_n.extend(list(ngrams(j.split(),1)))
        test_n=[]
        for j in test:
            test_n.extend(list(ngrams(j.split(),1)))
        freq_n=dict(FreqDist(train_n))
        for i in test_n:
            if i not in freq_n.keys():
                freq_n[i]=0
        for i in freq_n.keys():
            freq_n[i]+=1
        prob_n={}
        for i in freq_n.keys():
            #print(i)
            prob_n[i]=freq_n[i]/(len(train_n)+len(vocab))
        perplexityn=1
#         for token in test_n:
#             if token in prob_n.keys():
#                 perplexityn+=math.log(prob_n[token],2)
#             else:
#                 perplexityn+=math.log((1/(len(vocab)+len(test_n))),2)
#         perplexityn/=len(test_n)
#         perplexityn=2**(-perplexityn)
        #print(perplexityn)
       
        for token in test_n:
            #if token  in prob_n.keys():
            perplexityn*=((1/prob_n[token])**(1/len(test_n)))
            #else:
                #perplexityn*=((len(vocab)+len(train_n))**(1/len(test_n)))
            #perplexityn=perplexityn**(1/len(test))
            #print(perplexityn)
        #perplexity=perplexityn**(1/len(test))
        #print(perplexityn)
#         for token in test_n:
#             if token not in prob_n.keys():
#                 prob_n[token]=1/(len(train)+len(vocab))
#         for token in test_n:
#             if token  in prob_n.keys():
#                 perplexityn+=math.log(prob_n[token],2)
#             else:
#                 perplexityn+=math.log(1/(len(vocab)+len(train)))
#             print(perplexityn)
#         perplexityn/=len(test_n)
#         print("stage2",perplexityn)
#         perplexityn=2**(-perplexityn)
#         print("stage3",perplexityn)  
#         print(len(test_n))
            #perplexity=perplexity**(1/len(test_bi))
    return perplexityn, prob_n, freq_n1

        
    
    


In [269]:
#datangram function gives perxplexity and MLE of given model and ngram
def datangram(n,n_gram):
    perplexityn,probdist,freq_n1=ngram(n,train,test,vocab)
    if n>1:
        n_gram=tuple(n_gram.split())
        if n_gram in probdist.keys():
            print("perplexity= ",perplexityn, "MLE= ", probdist[n_gram])
        else:
            print("perplexity= ",perplexityn, "MLE= ", 1/len(vocab))
    else:
        if n_gram in probdist.keys():
            print("perplexity= ",perplexityn, "MLE= ", probdist[n_gram])
        else:
            print("perplexity= ",perplexityn, "MLE= ", 1/len(vocab))

In [260]:
#generator generates sentences of length 10
def Generator(n):
    import random
    if n>1:
        
        perplexityn,probdist, freq_n1=ngram(n,train,test,vocab)
        word = vocab[random.randint(0,len(vocab))]
        while type(word)!=str:
            word = vocab[random.randint(0,len(vocab))]
        gen=[word]
        for k in range(9):
            max=0
            #current=tuple()
            for tup in probdist.keys():
                if tup[0]==word:
                    #print(tup)
                    if probdist[tup]>max and tup[-1]!='</s>':
                        max=probdist[tup]
                        current=tup
                        #print(tup)
                        word=tup[1]
            #print(word)
            gen.append(word)
    else:
        gen=[]
        for k in range(10):
            gen.append(vocab[random.randint(0,len(vocab)-1)])
            
    return gen
                    
                    

In [261]:
#sentencegen generates required number of sentences from required model
def sentencegen(n_sentences,ngram):
    for i in range(n_sentences):
        print(Generator(ngram))

In [272]:
datangram(1,'here')

perplexity=  319.0917887003135 MLE=  0.00016471750947125678


In [273]:
datangram(2,'we are')

perplexity=  652.0499178505975 MLE=  0.015301750520259517


In [274]:
datangram(3,'thank you so')

perplexity=  2594.019840706311 MLE=  0.0004809233728759218


In [277]:
datangram(4,'make america great again')

perplexity=  4548.401831514004 MLE=  0.006707017830852282


In [262]:
sentencegen(5,1)

['wheres', 'badly', 'families', 'isis', "weren't", 'costs', 'reserve', 'turns', 'putt', 'prime']
['geniuses', 'film', 'rockets', 'crisis', 'at', 'troubled', 'shooter', 'doubled', 'prognosticator', 'generous']
['blow', 'swerve', 'leading', 'condos', 'blow', 'surface', 'suggested', 'headlines', 'options', 'same']
['referred', 'arizona', 'simplification', 'waiting', 'gulf', 'sell', 'won', 'upsidedown', 'subsidy', 'exchange']
['radicalized', 'shove', 'burst', 'petty', 'embarrassed', 'escaping', 'honoring', 'obvious', 'commerce', 'near']


In [263]:
sentencegen(5,2)

['though', 'wall', 'not', 'fair', 'guess', 'he', 'great', 'guy', 'talking', 'about']
['touch', 'them', 'been', 'that', 'mean', 'dont', 'hampshire', 'in', 'core', 'few']
['marginalizing', 'through', 'other', 'day', 'television', 'a', 'great', 'guy', 'talking', 'about']
['artificial', 'good', 'something', 'the', 'were', 'back', 'the', 'were', 'back', 'the']
['forced', 'guy', 'talking', 'about', 'of', 'and', 'i', 'to', 'know', 'it']


In [264]:
sentencegen(5,3)

['liar', 'anyways', 'free', 'trade', 'regain', 'friends', 'beginning', 'marco', 'great', 'respect']
['shame', 'were', 'right', 'old', 'would', 'this', 'look', 'supposed', 'pay', 'the']
['negotiations', 'rigged', 'illegal', 'want', 'our', 'country', 'headed', 'wrong', 'as', 'bad']
['incitement', 'movie', 'received', 'fight', 'year', 'out', 'of', 'other', 'day', 'other']
['wait', 'great', 'respect', 'steve', 'king', 'people', 'other', 'day', 'other', 'day']


In [279]:
sentencegen(5,4)

['early', 'of', 'not', 'repeal', 'take', 'hell', 'of', 'not', 'repeal', 'take']
['platform', "we're", 'take', 'hell', 'of', 'not', 'repeal', 'take', 'hell', 'of']
['nbc', 'thousands', 'environmental', 'too', 'better', 'best', 'country', 'hell', 'of', 'not']
['purchasers', 'didnt', 'important', 'isnt', 'about', 'whats', 'im', 'the', 'that', 'take']
['apologize', 'companies', 'wall', 'know', 'trade', 'mexico', 'kind', 'mindset', 'god', 'saying']


As it is observed, the output isn't very coherent. This can be attributed to the limitation of ngram wherein they cannot learn long term context